In [28]:
import pandas as pd
mutations = pd.read_csv('merged_variants_anno_vaf_filtered.txt', sep="\t",
                        header=0, index_col=0, low_memory=False)
metasamples = pd.read_excel('../Metadata2.xlsx')

In [29]:
# Keep only samples that are tumor-normal pair and labeled as included
df = metasamples.loc[(metasamples['INCLUDED'] == "Y")
                     & (metasamples['TUMOR_ONLY'] == "No")
                     & (metasamples['SAMPLE_TYPE'] != "gDNA"), 
                     ['PATIENT_ID', 'SAMPLE_ID']]
metasamples.index = metasamples['SAMPLE_ID']
meta_filtered = metasamples.reindex(df['SAMPLE_ID'].tolist())
SAMPLES = [x.strip() for x in df['SAMPLE_ID'] + '-' + df['PATIENT_ID']]
meta_filtered.index = SAMPLES
mutations.index = mutations['VARIANT_KEY']
mutations_filtered = mutations.loc[:, ['GENE'] + SAMPLES]

In [30]:
mutations_seen_E = list()
mutations_seen_F = list()

SAMPLES_E = meta_filtered[meta_filtered['VAR2'] == 'E'].index.to_list()
SAMPLES_F = meta_filtered[meta_filtered['VAR2'] == 'F'].index.to_list()
GENES = mutations_filtered['GENE']
for index, row in mutations_filtered.iloc[:,1:].iterrows():
    samples_E = list()
    samples_F = list()
    for column, value in row.items():
        if value >= 5.0:
            if column in SAMPLES_E:
                samples_E.append(column)
            if column in samples_F:
                samples_F.append(column)
    if len(samples_E) > 0 and len(samples_F) == 0:
        patients = [x.split('-')[-1] for x in samples_E]
        if len(set(patients)) > 1:
            mutations_seen_E.append((index,
                                     GENES[index],
                                     len(samples_E), 
                                     len(set([x.split('-')[-1] for x in samples_E]))))
    if len(samples_E) == 0 and len(samples_F) > 0:
        patients = [x.split('-')[-1] for x in samples_F]
        if len(set(patients)) > 1:
            mutations_seen_F.append((index,
                                     GENES[index],
                                     len(samples_F), 
                                     len(set([x.split('-')[-1] for x in samples_F]))))

In [31]:
import gseapy as gp
import matplotlib.pyplot as plt

genes_E = [x[1] for x in mutations_seen_E if x[3] > 1]
enr = gp.enrichr(gene_list = genes_E,
                 gene_sets = ['KEGG_2019_Human','GO_Biological_Process_2018', 'GO_Molecular_Function_2018'],
                 organism = 'Human',
                 description = 'Group E',
                 outdir = 'analysis_responders/E',
                 # no_plot=True,
                 cutoff = 1.0)
mutations.loc[[x[0] for x in mutations_seen_E if x[3] > 1], :].to_csv('analysis_responders/E/mutations_table.txt', sep="\t", header=True)

2021-02-02 15:12:14,483 Error fetching enrichment results: KEGG_2019_Human
2021-02-02 15:12:17,413 Error fetching enrichment results: GO_Biological_Process_2018
2021-02-02 15:12:20,494 Error fetching enrichment results: GO_Molecular_Function_2018


In [32]:
genes_F = [x[1] for x in mutations_seen_F if x[3] > 1]
enr = gp.enrichr(gene_list = genes_F,
                 gene_sets = ['KEGG_2019_Human','GO_Biological_Process_2018', 'GO_Molecular_Function_2018'],
                 organism = 'Human',
                 description = 'Group F',
                 outdir = 'analysis_responders/F',
                 # no_plot=True,
                 cutoff = 1.0)
mutations.loc[[x[0] for x in mutations_seen_F if x[3] > 1], :].to_csv('analysis_responders/F/mutations_table.txt', sep="\t", header=True)

Exception: Error analyzing gene list